### 4.4 自定义层

### 4.4.1 不含模型参数的自定义层

In [11]:
import torch
from torch import nn

In [12]:
class CenteredLayer(nn.Module):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()

In [13]:
layer = CenteredLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

In [14]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
y = net(torch.rand(4, 8))
# 接近于0
y.mean().item()

2.7939677238464355e-09

### 4.4.2 含模型参数的自定义层

> 1. **应该讲参数定义为`Parameter`类,这样参数可以被添加到模型的参数列表中**
> 2. **还可以使用`ParameterList`和`ParameterDict`**

In [26]:
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense, self).__init__()
        # 3个4 * 4
        self.params = nn.ParameterList([nn.Parameter(torch.rand(4, 4)) for _ in range(3)])
        # 1个 4 * 1
        self.params.append(nn.Parameter(torch.randn(4, 1)))
    
    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
    

In [27]:
net = MyListDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


In [19]:
# ParameterDict 使用update() 新增参数, 使用keys()返回所有键,使用item()返回所有键值对
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
            'linear1': nn.Parameter(torch.randn(4, 4)),
            'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4, 2))})
    
    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])

In [20]:
net = MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [23]:
# size (1, 4)
x = torch.ones(1, 4)
print(net(x, 'linear1'))
print(net(x, 'linear2'))
print(net(x, 'linear3'))

tensor([[-6.2501,  1.0591,  2.1303,  0.4649]], grad_fn=<MmBackward>)
tensor([[2.8337]], grad_fn=<MmBackward>)
tensor([[ 1.6498, -1.7078]], grad_fn=<MmBackward>)


In [28]:
net = nn.Sequential(
    MyDictDense(),
    MyListDense()
)
print(net)
print(net(x))

Sequential(
  (0): MyDictDense(
    (params): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): MyListDense(
    (params): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
)
tensor([[0.0803]], grad_fn=<MmBackward>)
